In [1]:
# pip install --upgrade --quiet  langchain langchain-community langchain-experimental langchain-openai

In [2]:
import os
os.environ['OPENAI_API_KEY'] = ''

In [3]:
from langchain_community.utilities import SQLDatabase

# Replace these with your actual database credentials
username = 'your_username'
password = 'your_password'
host = 'localhost'  # or the IP address of your MySQL server
database = 'your_database_name'

db = SQLDatabase.from_uri(f'mysql+mysqlconnector://{username}:{quote(password)}@{host}:3306/{database}' , sample_rows_in_table_info=3)
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['candidate_card', 'candidate_local_db', 'jd_candidate_ranks', 'jd_history', 'jd_task_choose', 'mgr_mappings']


### Few-shot examples

In [4]:
examples = [
    {
        "input": "Give me the list of all candidates with active email or phone numbers.",
        "query": "SELECT name, email, phone_number FROM candidate_card WHERE email IS NOT NULL OR phone_number IS NOT NULL;",
    },
    {
        "input": "List of all candidates with high relevance.",
        "query": "SELECT name, jd_match_score FROM candidate_card ORDER BY jd_match_score DESC LIMIT 10;",
    },
    {
        "input": "Give me the contact information of candidate Abhinav Nayak.",
        "query": "SELECT `Candidate Name`, `Email`, `Phone Number`, `location` FROM candidate_local_db WHERE `Candidate Name` = 'Abhinav Nayak';",
    },
    {
        "input": "List all Candidates from Hyderabad.",
        "query": "SELECT `Candidate Name`, `location` FROM candidate_local_db WHERE `location` LIKE '%Hyderabad%';",
    },
    {
        "input": "List of all the candidates having skill Java",
        "query": "SELECT `Candidate Name`, skills FROM candidate_local_db WHERE skills LIKE '%Java%';",
    },
    {
        "input": "What is the status of Java Developer JD",
        "query": "SELECT  jd_name, company_name, Contacted, Selected, Rejected, status FROM jd_history WHERE jd_name LIKE '%JavaDeveloper%';",
    },
    {
        "input": "Give all the names of all candidates added to Java Developer",
        "query": "SELECT candidate_card.name FROM candidate_card JOIN jd_history ON jd_history.jd_id = candidate_card.jd_id WHERE candidate_card.jd_id IN (SELECT jd_id FROM jd_history WHERE jd_name LIKE '%JavaDeveloper%');",
    },
    {
        "input": "How many are highly relevant to this JD ?",
        "query": """SELECT candidate_card.name, candidate_card.jd_match_score FROM candidate_card JOIN jd_history ON candidate_card.jd_id = jd_history.jd_id WHERE jd_history.jd_name LIKE 'software engineer' ORDER BY candidate_card.jd_match_score DESC LIMIT 20;"""
    },
    {
        "input": "Who is working on this JD X?",
        "query": """SELECT fullname FROM jd_history WHERE jd_name LIKE '%X%' LIMIT 3"""
    },
    {
        "input": "What is the status of Candidate X for JD Y?",
        "query": """SELECT `name`, jd_match_score, is_interested, `status`, status_name FROM candidate_card WHERE name = 'X' and jd_id = ( SELECT jd_id from jd_history WHERE jd_name = 'Y')"""
    },
    {
        "input": "Top 5 candidate for jd X",
        "query": """SELECT name, jd_match_score FROM candidate_card WHERE jd_id = (SELECT jd_id from jd_history WHERE jd_name = 'X') Order by jd_match_score desc LIMIT 5;"""
    }
]

# examples = [
#     {"input": "",
#      "query": "",
#     }
# ]

### Dynamic few-shot examples

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["input"],
)

In [6]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

In [7]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

### Agent 

In [8]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [9]:
from langchain.agents import AgentType

agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
    agent_executor_kwargs={"return_intermediate_steps": True}
)

In [10]:
response= agent.invoke({"input": "Get all the candidate information who knows AI/ML"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': "SELECT `Candidate Name`, skills, experience FROM candidate_local_db WHERE skills LIKE '%AI%' OR skills LIKE '%ML%'"}`


Error: (sqlite3.OperationalError) no such column: experience
[SQL: SELECT `Candidate Name`, skills, experience FROM candidate_local_db WHERE skills LIKE '%AI%' OR skills LIKE '%ML%']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Invoking: `sql_db_schema` with `{'table_names': 'candidate_local_db'}`



CREATE TABLE candidate_local_db (
	cid BIGINT, 
	company_id BIGINT, 
	cloud_processing_id BIGINT, 
	"Candidate Name" TEXT, 
	"Phone Number" TEXT, 
	"Email" TEXT, 
	location TEXT, 
	location_lat_longs TEXT, 
	location_states TEXT, 
	location_country TEXT, 
	earlier_locations TEXT, 
	earlier_lat_longs TEXT, 
	earlier_loc_states TEXT, 
	earlier_loc_country TEXT, 
	grad_years TEXT, 
	"CV Link" TEXT, 
	"Last Company" TEXT, 
	"Education" TEXT, 
	"Profile Image" TEXT, 
	status

In [11]:
n = len(response["intermediate_steps"])
for i in range(0,n,2):
    print(response["intermediate_steps"][i][0].tool_input, end="\n\n")
    if(i!=n-1): print(response["intermediate_steps"][i][1], end="\n\n")

{'query': "SELECT `Candidate Name`, skills, experience FROM candidate_local_db WHERE skills LIKE '%AI%' OR skills LIKE '%ML%'"}

Error: (sqlite3.OperationalError) no such column: experience
[SQL: SELECT `Candidate Name`, skills, experience FROM candidate_local_db WHERE skills LIKE '%AI%' OR skills LIKE '%ML%']
(Background on this error at: https://sqlalche.me/e/20/e3q8)

{'query': "SELECT `Candidate Name`, skills FROM candidate_local_db WHERE skills LIKE '%AI%' OR skills LIKE '%ML%'"}



In [12]:
response = agent.invoke({"input": "what is the status of java jd"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': "SELECT jd_name, company_name, Contacted, Selected, Rejected, status FROM jd_history WHERE jd_name LIKE '%Java%' LIMIT 10"}`


[('Morgan_Optimum_AEM+Java_', 'Morgan Stanley', 0, 0, 0, 1), ('Sr_Java_Dev_shiva', 'Tarento Technologies', 0, 0, 0, 1), ('java_developer_optimum', 'Client: Morgan Stanley - Company; OptimumSolutions', 0, 0, 0, 1), ('Java', '10x', 0, 0, 0, 1), ('Java Developer', 'Tarento', 0, 0, 0, 1), ('java_dev_tarento', 'Tarento Technologies', 0, 0, 0, 1), ('java_persistent', 'Persistent Systems', 0, 0, 0, 1), ('Java-PSE', 'Hiretale', 0, 0, 0, 1), ('EY Java Dev', 'EY', 0, 0, 0, 1), ('EY Java Dev11', 'EY', 0, 0, 0, 1)]The status of Java JDs are as follows:
1. JD Name: Morgan_Optimum_AEM+Java, Company: Morgan Stanley, Contacted: 0, Selected: 0, Rejected: 0, Status: 1
2. JD Name: Sr_Java_Dev_shiva, Company: Tarento Technologies, Contacted: 0, Selected: 0, Rejected: 0, Status: 1
3. JD Name: jav

In [13]:
n = len(response["intermediate_steps"])
for i in range(0,n,2):
    print(response["intermediate_steps"][i][0].tool_input, end="\n\n")
    if(i!=n-1): print(response["intermediate_steps"][i][1], end="\n\n")

{'query': "SELECT jd_name, company_name, Contacted, Selected, Rejected, status FROM jd_history WHERE jd_name LIKE '%Java%' LIMIT 10"}



In [14]:
response = agent.invoke({"input": "who won yesterday ipl match "})



> Entering new SQL Agent Executor chain...
I don't know.

> Finished chain.


In [15]:
n = len(response["intermediate_steps"])
for i in range(0,n,2):
    print(response["intermediate_steps"][i][0].tool_input, end="\n\n")
    if(i!=n-1): print(response["intermediate_steps"][i][1], end="\n\n")

In [16]:
response = agent.invoke({"input": "List of candidates added to java jd"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': "SELECT candidate_card.name FROM candidate_card JOIN jd_history ON candidate_card.jd_id = jd_history.jd_id WHERE jd_history.jd_name LIKE '%Java%' LIMIT 10"}`


[('Abhay Vishwakarma',), ('Abhishek Desai',), ('Abhishek Tiwari',), ('Abid Hussain Shaikh',), ('Aditya Gupta',), ('Ajay Sawale',), ('Akshay Adav',), ('Akshay Kotecha',), ('Akshay Sachan',), ('Akshay Shinde',)]The list of candidates added to the Java JD includes:
1. Abhay Vishwakarma
2. Abhishek Desai
3. Abhishek Tiwari
4. Abid Hussain Shaikh
5. Aditya Gupta
6. Ajay Sawale
7. Akshay Adav
8. Akshay Kotecha
9. Akshay Sachan
10. Akshay Shinde

> Finished chain.


In [17]:
n = len(response["intermediate_steps"])
for i in range(0,n,2):
    print(response["intermediate_steps"][i][0].tool_input, end="\n\n")
    if(i!=n-1): print(response["intermediate_steps"][i][1], end="\n\n")

{'query': "SELECT candidate_card.name FROM candidate_card JOIN jd_history ON candidate_card.jd_id = jd_history.jd_id WHERE jd_history.jd_name LIKE '%Java%' LIMIT 10"}



In [18]:
response = agent.invoke({"input": "Who is working on this java jd ?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': "SELECT fullname FROM jd_history WHERE jd_name LIKE '%java%' LIMIT 10"}`


Error: (sqlite3.OperationalError) no such table: jd_history
[SQL: SELECT fullname FROM jd_history WHERE jd_name LIKE '%java%' LIMIT 10]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Invoking: `sql_db_list_tables` with `{}`


candidate_card, candidate_local_db, jd_candidate_ranks, jd_history, jd_task_choose, mgr_mappings
Invoking: `sql_db_query` with `{'query': "SELECT fullname FROM jd_history WHERE jd_name LIKE '%java%' LIMIT 10"}`


[('1a',), ('user1_zmartt',), ('1a',), ('1a',), ('user1_zmartt',), ('user1_zmartt',), ('User Hiretale',), ('User Hiretale',), ('User Hiretale',), ('User Hiretale',)]The individuals working on the Java JD are: 1a, user1_zmartt, User Hiretale.

> Finished chain.


In [19]:
n = len(response["intermediate_steps"])
for i in range(0,n,2):
    print(response["intermediate_steps"][i][0].tool_input, end="\n\n")
    if(i!=n-1): print(response["intermediate_steps"][i][1], end="\n\n")

{'query': "SELECT fullname FROM jd_history WHERE jd_name LIKE '%java%' LIMIT 10"}

Error: (sqlite3.OperationalError) no such table: jd_history
[SQL: SELECT fullname FROM jd_history WHERE jd_name LIKE '%java%' LIMIT 10]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

{'query': "SELECT fullname FROM jd_history WHERE jd_name LIKE '%java%' LIMIT 10"}

